## Deliverable 2. Create a Customer Travel Destinations Map.

In [4]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
from pprint import pprint

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [5]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Aksarka,RU,2021-12-22 15:1032,66.5606,67.7975,8.08,96,100,7.96,light snow
1,1,Qasigiannguit,GL,2021-12-22 15:1032,68.8193,-51.1922,39.45,51,96,20.18,overcast clouds
2,2,Grindavik,IS,2021-12-22 15:1032,63.8424,-22.4338,36.07,87,90,8.05,overcast clouds
3,3,Mount Gambier,AU,2021-12-22 15:1033,-37.8333,140.7667,48.97,89,72,4.76,broken clouds
4,4,Ponta Delgada,PT,2021-12-22 15:1033,37.7333,-25.6667,60.49,72,20,27.63,few clouds


In [6]:
len(city_data_df)

708

In [7]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip?"))
max_temp = float(input("What is the maximum temperature you would like for your trip?"))

What is the minimum temperature you would like for your trip?75
What is the maximum temperature you would like for your trip?90


In [8]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
#Filter the data set to find the cities that fit the criteria
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & 
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
8,8,Ambulu,ID,2021-12-22 15:1034,-8.3450,113.6058,78.71,85,87,2.68,overcast clouds
9,9,Ambilobe,MG,2021-12-22 15:1034,-13.2000,49.0500,83.28,73,54,5.44,light rain
10,10,Atuona,PF,2021-12-22 15:0721,-9.8000,-139.0333,78.78,73,91,20.78,overcast clouds
11,11,Amapa,BR,2021-12-22 15:1034,1.0000,-52.0000,83.57,71,100,6.53,overcast clouds
17,17,Avarua,CK,2021-12-22 15:1035,-21.2078,-159.7750,75.25,83,100,9.22,moderate rain
18,18,Rikitea,PF,2021-12-22 15:1035,-23.1203,-134.9692,76.68,80,86,10.45,overcast clouds
19,19,Gondanglegi,ID,2021-12-22 15:1036,-7.7479,112.0132,76.89,92,86,1.88,overcast clouds
21,21,Songkhla,TH,2021-12-22 15:1036,6.8333,100.6667,77.05,93,70,1.21,broken clouds
25,25,Saint-Philippe,RE,2021-12-22 15:1037,-21.3585,55.7679,78.35,88,100,16.42,light rain
28,28,Mozarlandia,BR,2021-12-22 15:1038,-14.7447,-50.5706,82.15,78,88,6.64,overcast clouds


In [9]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                1
Date                   0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [10]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.

clean_df = preferred_cities_df.dropna()
clean_df

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
8,8,Ambulu,ID,2021-12-22 15:1034,-8.3450,113.6058,78.71,85,87,2.68,overcast clouds
9,9,Ambilobe,MG,2021-12-22 15:1034,-13.2000,49.0500,83.28,73,54,5.44,light rain
10,10,Atuona,PF,2021-12-22 15:0721,-9.8000,-139.0333,78.78,73,91,20.78,overcast clouds
11,11,Amapa,BR,2021-12-22 15:1034,1.0000,-52.0000,83.57,71,100,6.53,overcast clouds
17,17,Avarua,CK,2021-12-22 15:1035,-21.2078,-159.7750,75.25,83,100,9.22,moderate rain
...,...,...,...,...,...,...,...,...,...,...,...
699,699,Watsa,CD,2021-12-22 15:2835,3.0372,29.5355,78.62,50,100,2.10,overcast clouds
701,701,Kitgum,UG,2021-12-22 15:2836,3.2783,32.8867,87.49,31,99,12.53,overcast clouds
703,703,Tahoua,NE,2021-12-22 15:2836,14.8888,5.2692,85.14,13,54,11.48,broken clouds
705,705,Salinopolis,BR,2021-12-22 15:2837,-0.6136,-47.3561,85.73,68,91,14.83,overcast clouds


In [11]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
8,Ambulu,ID,78.71,overcast clouds,-8.3450,113.6058,
9,Ambilobe,MG,83.28,light rain,-13.2000,49.0500,
10,Atuona,PF,78.78,overcast clouds,-9.8000,-139.0333,
11,Amapa,BR,83.57,overcast clouds,1.0000,-52.0000,
17,Avarua,CK,75.25,moderate rain,-21.2078,-159.7750,
18,Rikitea,PF,76.68,overcast clouds,-23.1203,-134.9692,
19,Gondanglegi,ID,76.89,overcast clouds,-7.7479,112.0132,
21,Songkhla,TH,77.05,broken clouds,6.8333,100.6667,
25,Saint-Philippe,RE,78.35,light rain,-21.3585,55.7679,
28,Mozarlandia,BR,82.15,overcast clouds,-14.7447,-50.5706,


In [12]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    hotel_loc = f"{lat},{lng}"
    params['location']= hotel_loc
    
    #Use the search term: "lodging" and our latitude and longitude
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url,  params = params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    
    try:
        hotel_df.loc[index,"Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
 

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [13]:

# 7. Drop the rows where there is no Hotel Name.
null_hotels = hotel_df.loc[hotel_df["Hotel Name"]==""]
hotels_to_drop = null_hotels.index
clean_hotel_df = hotel_df.drop(index = hotels_to_drop)
clean_hotel_df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 210 entries, 8 to 706
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   City                 210 non-null    object 
 1   Country              210 non-null    object 
 2   Max Temp             210 non-null    float64
 3   Current Description  210 non-null    object 
 4   Lat                  210 non-null    float64
 5   Lng                  210 non-null    float64
 6   Hotel Name           210 non-null    object 
dtypes: float64(3), object(4)
memory usage: 13.1+ KB


In [14]:
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
8,Ambulu,ID,78.71,overcast clouds,-8.3450,113.6058,Hotel Ambulu Jember
9,Ambilobe,MG,83.28,light rain,-13.2000,49.0500,Hôtel & Restaurant DIANA Ambilobe
10,Atuona,PF,78.78,overcast clouds,-9.8000,-139.0333,Villa Enata
17,Avarua,CK,75.25,moderate rain,-21.2078,-159.7750,Paradise Inn
18,Rikitea,PF,76.68,overcast clouds,-23.1203,-134.9692,People ThankYou


In [20]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPY_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")
clean_hotel_df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 210 entries, 8 to 706
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   City                 210 non-null    object 
 1   Country              210 non-null    object 
 2   Max Temp             210 non-null    float64
 3   Current Description  210 non-null    object 
 4   Lat                  210 non-null    float64
 5   Lng                  210 non-null    float64
 6   Hotel Name           210 non-null    object 
dtypes: float64(3), object(4)
memory usage: 13.1+ KB


In [16]:
# 9. Using the template add city name, the country code, the current description and maximum temperature for the city.

info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} F</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
</dl>
"""
# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]



In [17]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations,info_box_content = hotel_info)
fig = gmaps.figure(center = (30.0, 31.0), zoom_level = 1.5)
fig.add_layer(marker_layer)
# 11b. Display the figure

fig

Figure(layout=FigureLayout(height='420px'))

In [18]:
clean_hotel_df.head(30)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
8,Ambulu,ID,78.71,overcast clouds,-8.3450,113.6058,Hotel Ambulu Jember
9,Ambilobe,MG,83.28,light rain,-13.2000,49.0500,Hôtel & Restaurant DIANA Ambilobe
10,Atuona,PF,78.78,overcast clouds,-9.8000,-139.0333,Villa Enata
17,Avarua,CK,75.25,moderate rain,-21.2078,-159.7750,Paradise Inn
18,Rikitea,PF,76.68,overcast clouds,-23.1203,-134.9692,People ThankYou
19,Gondanglegi,ID,76.89,overcast clouds,-7.7479,112.0132,Rumah Mujiatin
21,Songkhla,TH,77.05,broken clouds,6.8333,100.6667,น.ส ซัลมา สาแม็ง
25,Saint-Philippe,RE,78.35,light rain,-21.3585,55.7679,"Chambres d'hôte ""La Trinité"""
28,Mozarlandia,BR,82.15,overcast clouds,-14.7447,-50.5706,Hotel Santa Mônica
31,Castro,BR,86.02,clear sky,-24.7911,-50.0119,CHACARA BAILLY
